In [9]:
import geopandas as gpd
from pathlib import Path
import numpy as np
import pandas as pd

import pickle
import src.modeling_turns as modeling_turns
import src.add_attributes as add_attributes
import src.prepare_network as prepare_network

import sys
sys.path.insert(0,str(Path.cwd().parent))
import file_structure_setup
config = file_structure_setup.filepaths()

In [10]:
unfiltered_turns = pd.read_parquet(config['network_fp']/'turns_df.parquet')
unfiltered_directed_links = pd.read_parquet(config['network_fp']/'directed_edges.parquet')
unfiltered_links = gpd.read_file(config['network_fp']/'final_network.gpkg',layer='edges')
unfiltered_nodes = gpd.read_file(config['network_fp']/'final_network.gpkg',layer='nodes')

In [11]:
# Remove wrongway from directed links and turns
oneway_dict = dict(zip(unfiltered_links['linkid'],unfiltered_links['oneway']))
unfiltered_directed_links['wrongway'] = (unfiltered_directed_links['linkid'].map(oneway_dict) == True) & (unfiltered_directed_links['reverse_link']==True)
directed_links = unfiltered_directed_links[unfiltered_directed_links['wrongway']==False]
unfiltered_turns['source_oneway'] = unfiltered_turns['source_linkid'].map(oneway_dict)
unfiltered_turns['target_oneway'] = unfiltered_turns['target_linkid'].map(oneway_dict)
source_wrongway = ((unfiltered_turns['source_oneway'] == True) & (unfiltered_turns['source_reverse_link'] == True)) == False
target_wrongway = ((unfiltered_turns['target_oneway'] == True) & (unfiltered_turns['target_reverse_link'] == True)) == False
unfiltered_turns = unfiltered_turns[source_wrongway & target_wrongway]

In [12]:
print(unfiltered_links.shape[0],'links')
node_count = unfiltered_links['A'].append(unfiltered_links['B']).nunique()
print(unfiltered_directed_links.shape[0],'directed links')
print(node_count,'nodes')
print(unfiltered_turns.shape[0],'turns')

107397 links
214794 directed links
91820 nodes
350746 turns


# Filtered Network

In [13]:
#export calibration network
with (config['calibration_fp']/"calibration_network.pkl").open('rb') as fh:
    links, turns = pickle.load(fh)
undirected_links = links[links['reverse_link']==False]

In [14]:
print(links['linkid'].nunique(),'links')
node_count = links['A'].append(links['B']).nunique()
print(links.shape[0],'directed links')
print(node_count,'nodes')
print(turns.shape[0],'turns')

59112 links
111192 directed links
56084 nodes
149047 turns


Undirected Attributes

In [15]:
undirected_links.to_file(config['network_fp']/'network_stats.gpkg',layer='undirected_attributes')

In [16]:
undirected_links['length_mi']

0         0.53
2         0.63
4         0.58
7         0.31
9         0.33
          ... 
118214    0.03
118216    0.04
118219    0.09
118221    0.08
118222    0.13
Name: length_mi, Length: 59112, dtype: float64

In [17]:
miles = undirected_links.groupby('lanes')['length_mi'].sum().round(0)
counts = undirected_links['lanes'].value_counts()
pct = (miles / undirected_links['length_mi'].sum() * 100).round(0)
pct.name = 'pct'
pd.concat([counts,miles,pct],axis=1)

,lanes,length_mi,pct
1,45151,1673.0,78.0
0,8922,225.0,10.0
2,4533,233.0,11.0
3,506,22.0,1.0


In [18]:
undirected_links.loc[undirected_links['speed']==0,'speed'] = 1

In [19]:
miles = undirected_links.groupby('speed')['length_mi'].sum().round(0)
counts = undirected_links['speed'].value_counts()
pct = (miles / undirected_links['length_mi'].sum() * 100).round(0)
pct.name = 'pct'
pd.concat([counts,miles,pct],axis=1)

,speed,length_mi,pct
1,59112,2152.0,100.0


In [20]:
undirected_links['speed'].value_counts()

1    59112
Name: speed, dtype: int64

In [21]:
undirected_links['AADT'].describe()

count    59112.000000
mean      3407.183059
std       6801.933972
min          0.000000
25%          0.000000
50%       2107.000000
75%       2107.000000
max      63900.000000
Name: AADT, dtype: float64

In [22]:
undirected_links['truck_pct'].describe()

count    59112.000000
mean         0.637722
std          4.225946
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        270.000000
Name: truck_pct, dtype: float64

Directed

In [23]:
min_grade = links.groupby('linkid')['ascent_grade_%'].idxmax()
links.loc[min_grade,['linkid','name','highway','ascent_grade_%','geometry']].to_file(config['network_fp']/'network_stats.gpkg',layer='elevation')

In [24]:
ranking = {
    'multi use path':0,
    'bike lane':3,
    'buffered bike lane':2,
    'cycletrack':1
}
bicycle_facilities = links.loc[links['facility_fwd'].notna(),['linkid','name','highway','facility_fwd','geometry']].copy()
bicycle_facilities = bicycle_facilities[bicycle_facilities.drop(columns=['geometry']).duplicated()==False]
bicycle_facilities['facility_ranking'] = bicycle_facilities['facility_fwd'].map(ranking)
bicycle_facilities_idx = bicycle_facilities.groupby('linkid')['facility_ranking'].idxmin()
bicycle_facilities = bicycle_facilities.loc[bicycle_facilities_idx]
bicycle_facilities.to_file(config['network_fp']/'network_stats.gpkg',layer='bicycle_facilities')